[View source on GitHub]: https://github.com/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/move_device.ipynb
[Notebook Viewer]: https://nbviewer.jupyter.org/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/move_device.ipynb
[Run in binder]: https://mybinder.org/v2/gh/wadmp/wadmp.github.io/master?filepath=jupyter_notebooks%2Fmove_device.ipynb
[Run in Google Colab]: https://colab.research.google.com/github/wadmp/wadmp.github.io/blob/master/jupyter_notebooks/move_device.ipynb

| [![GitHub logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/github_logo.png)][View source on GitHub] | [![Jupyter logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/jupyter_logo.png)][Notebook Viewer] | [![binder logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/binder_logo.png)][Run in binder] | [![Colab logo](https://raw.githubusercontent.com/wadmp/wadmp.github.io/master/images/colab_logo.png)][Run in Google Colab] |
|:---------------------:|:---------------:|:-------------:|:-------------------:|
| [View source on GitHub] | [Notebook Viewer] | [Run in binder] | [Run in Google Colab] |

## Introduction
This notebook allows you to move a device from one WebAccess/DMP Management Server ("Server 1") to another WebAccess/DMP Management Server ("Server 2").

> For background information, see [How to move a device to another Management Server](https://docs.wadmp.com/how-tos/move-a-device.md)

It does several things:
1. Identify what Bootstrap Server the device is using. This is usually Server 1, but it may not be. We call it Server 0.
1. Create the device on Server 2; (if it is there already, that is not a problem)
2. Configure the TLS "trust store" on Server 2 to trust devices that present a certificate that was signed by Server 0;
3. Configure the Bootstrap Server on Server 0 to direct the device to Server 2;
4. Trigger the device (via Server 1) to Bootstrap again.

### Requirements
* You need to have an existing user account on *both* WA/DMP servers.
* The device in question must exist on Server 1.
  * It does not necessarily need to be currently online on Server 1, but if it is not online then obviously you won't be able to confirm if the script has worked!
* On Server 0, you must have the "Device Management Server" permission.
* On Server 2, you must be a SysAdmin!

### Usage
Run the cells, either one at a time (Shift-Enter is your friend!), or all at once.

When prompted, enter the required User Input.

> Note that when entering the URL of each WA/DMP server, you should use the URL that you use in your web browser,
> when viewing the "User Interface" of WebAccess/DMP. e.g. "wadmp.com" or "dev.wadmp.com".
> The URL for the corresponding API gateway will be calculated automatically: https://gateway.wadmp.com or https://gateway.dev.wadmp.com.

## Setup

In [ ]:
%%capture

# Install packages in the current Jupyter kernel
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pyjwt
!{sys.executable} -m pip install cryptography

import os
from urllib.parse import urlparse, urlunparse
import requests
import json
import base64
import re
import jwt
from cryptography import x509
from cryptography.x509.oid import NameOID
from cryptography.hazmat.backends import default_backend

## Global variables

In [ ]:
BASE_PATH = 'api'
SCHEME = 'https'
SESSION1 = requests.Session()  # Use one HTTPS session for all API calls to Server 1
SESSION2 = requests.Session()  # Use one HTTPS session for all API calls to Server 2

## Functions to be used later

In [ ]:
def login(base_url, session, username, password):
    """Login to the system, and return a token
    """
    url = f"{base_url}/public/auth/connect/token"
    credentials = {'username': username, 'password': password, 'client_id': 'python', 'grant_type': 'password'}
    print(f"Sending POST request to {url} with:\n"
          f"    credentials={credentials}\n")
    response = session.post(url, data=credentials)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()["access_token"]
    else:
        print("Failed to login!")
        sys.exit(1)


def get_devices(base_url, session, page_number, page_size, name):
    """Retrieves one page of the list of your devices.
       name = Optional filter for device’s alias.
    """
    url = f"{base_url}/{BASE_PATH}/management/devices"

    # The only REQUIRED query parameters are page and pageSize
    print(f"Sending GET request to {url} with:\n"
          f"    name={name}\n"
          f"    page={page_number}\n"
          f"    pageSize={page_size}")
    query = {'name': name, 'page': page_number, 'pageSize': page_size}
    response = session.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)
    
    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print(f"Failed to retrieve page {page_number} of devices!")
        return None


def get_apps_in_device(base_url, session, mac):
    """Gets apps installed in a device.
    """
    url = f"{base_url}/{BASE_PATH}/management/devices/{mac}/apps"

    print(f"Sending GET request to {url}:\n")
    response = session.get(url)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)
    
    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print(f"Failed to retrieve list of apps!")
        return None


def get_app_version_id(app_name, list_of_apps):
    """Get application version ID from a list of apps, such as that returned by get_apps_in_device()
    """
    for app in list_of_apps:
        if app["application_version"]["application"]["name"] == app_name:
            return app["application_version"]["id"]

    return None


def get_app_configuration(base_url, session, mac, app_name):
    """Get the desired and reported settings (of all sections) of an app in a device.
    """
    apps = get_apps_in_device(base_url, session, mac)
    app_version_id = get_app_version_id(app_name, apps)
    
    url = f"{base_url}/{BASE_PATH}/management/devices/{mac}/apps/{app_version_id}/settings"

    print(f"Sending GET request to {url}:\n")
    response = session.get(url)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)
    
    if response.status_code == requests.codes['ok']:
        return response.json()['data'][0]
    else:
        print(f"Failed to retrieve the app configuration!")
        return None
    

def ini_to_dict(ini_string):
    """Convert an INI-format string to a dictionary.
    
    e.g.
    "MOD_WADMP_CLIENT_ENABLED=1\nMOD_WADMP_CLIENT_BOOTSTRAPSERVER=bootstrap.wadmp.com\nMOD_WADMP_CLIENT_BOOTSTRAPPORT=8884\nMOD_WADMP_CLIENT_LOGLEVEL=debug\nMOD_WADMP_CLIENT_MONITORING_ENABLED=1\nMOD_WADMP_CLIENT_MONITORING_INTERVAL=15\n"
    becomes
    {
        MOD_WADMP_CLIENT_ENABLED: 1
        MOD_WADMP_CLIENT_BOOTSTRAPSERVER: bootstrap.wadmp.com
        MOD_WADMP_CLIENT_BOOTSTRAPPORT: 8884
        MOD_WADMP_CLIENT_LOGLEVEL: debug
        MOD_WADMP_CLIENT_MONITORING_ENABLED: 1
        MOD_WADMP_CLIENT_MONITORING_INTERVAL: 15
    }
    """
    d = {}
    for config_item in ini_string.splitlines():
        key, value = config_item.split('=')
        d[key]=value
    
    return d


def create_device(base_url, session, model=None):
    """Create a device in the system.
    """
    url = f"{base_url}/{BASE_PATH}/identity/devices"
    print(f"Sending POST request to {url} with:\n"
          f"    model={model}\n")
    response = session.post(url, json=model)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']['id']
    else:
        print.error("Failed to create device!")
        return None


def get_ms(base_url, session, Name=None, Address=None):
    """Gets the list of Management Server definition in the system.
    """
    url = f"{base_url}/{BASE_PATH}/bootstrap"
    print(f"Sending GET request to {url} with:\n"
          f"    Name={Name}\n"
          f"    Address={Address}\n")
    query = {'Name': Name, 'Address': Address}
    response = session.get(url, params=query)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']
    else:
        print("Failed to retrieve the list of Management Servers!")
        return None


def create_ms(base_url, session, name, address, port, company_id):
    """Creates a new Management Server definition in the system.
    
       name: is not important: it serves as a label or reminder for the user only.
       address: is the IP address or DNS name of the Management Server.
       port: is the TCP port number of the Management Server. (Usually 8883, which is the standard port number for MQTT over TLS.)
       company_id: is your company ID in the system.
    """
    url = f"{base_url}/{BASE_PATH}/bootstrap"
    print(f"Sending POST request to {url} with:\n"
          f"    name={name}\n"
          f"    address={address}\n"
          f"    port={port}\n"
          f"    company_id={company_id}\n")
    body = {'name': name, 'address': address, 'port': port, 'company_id': company_id}
    response = session.post(url, json=body)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']['id']
    else:
        print("Failed to create new MS definition!")
        return None


def use_ms(base_url, session, mac, ms_id):
    """Configure a device to use a new Management Server.
       This will only take effect when the device Bootstraps again.
    """
    url = f"{base_url}/{BASE_PATH}/management/devices/{mac}/bootstrap-server/{ms_id}"
    print(f"Sending PUT request to {url}\n")
    response = session.put(url)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['success']
    else:
        print("Failed to configure device to use MS!")
        return None


def trigger_bootstrap(base_url, session, mac):
    """Sends a command to trigger bootstrap process on a device.
    """
    url = f"{base_url}/{BASE_PATH}/management/devices/{mac}/commands/trigger-bootstrap"
    print(f"Sending POST request to {url}\n")
    response = session.post(url)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['success']
    else:
        print.error("Failed to trigger bootstrap!")
        return None


def get_certs(base_url, session):
    """Gets the device CA certificates in the trust store of the Management Server.
       The returned value is a Base64-encoded string of all of the trusted CA certs, in PEM format.
    """
    url = f"{base_url}/{BASE_PATH}/certs"
    print(f"Sending GET request to {url}\n")
    response = session.get(url)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['data']['certs']
    else:
        print("Failed to retrieve the certificates!")
        return None


def put_certs(base_url, session, certs):
    """Replaces the device CA certificates in the trust store of the Management Server.
       certs = Base64-encoded string, comprising ALL trusted CA certificates in PEM format.
    """
    url = f"{base_url}/{BASE_PATH}/certs"
    print(f"Sending PUT request to {url} with:\n"
          f"    certs={certs}\n")
    body = {'certs': certs}
    response = session.put(url, json=body)

    print(response.status_code)
    try:
        print(json.dumps(response.json(), indent=4, sort_keys=True))
    except ValueError:
        print(response.text)

    if response.status_code == requests.codes['ok']:
        return response.json()['success']
    else:
        print("Failed to edit MS definition!")
        return None

## User input
Reference: https://docs.python.org/3/library/urllib.parse.html#url-parsing

In [ ]:
device_name = input("Enter device alias or MAC address:")

SERVER1 = input("Enter the URL of Server 1 (where the device is currently connected. E.g. 'dev.wadmp.com'):")
USERNAME1 = input("Username on Server 1:")
PASSWORD1 = input("Password on Server 1:")

# If the URL for the UI is "dev.wadmp.com", the URL for the API Gateway is "https://gateway.dev.wadmp.com"
url = urlparse(f"{SCHEME}://{SERVER1}")
new_netloc = f"gateway.{url.netloc}"
GATEWAY1 = urlunparse(url._replace(netloc=new_netloc))
new_netloc = f"bootstrap.{url.netloc}"
BS1 = urlunparse(url._replace(netloc=new_netloc))

# If the URL for the UI is "https://dev.wadmp.com",
# the URL for the Bootstrap Server is "bootstrap.dev.wadmp.com"
BS1 = f"bootstrap.{url.netloc}"
# and the Management Server is "management.dev.wadmp.com"
MS1 = f"management.{url.netloc}"

SERVER2 = input("Enter the URL of Server 2 (where you want the device to connect to next):")
USERNAME2 = input("Username of SysAdmin on Server 2:")
PASSWORD2 = input("Password on Server 2:")

# If the URL for the UI is "https://wadmp.com", the URL for the API Gateway is "https://gateway.wadmp.com"
url = urlparse(f"{SCHEME}://{SERVER2}")
new_netloc = f"gateway.{url.netloc}"
GATEWAY2 = urlunparse(url._replace(netloc=new_netloc))

# If the URL for the UI is "https://wadmp.com",
# the URL for the Bootstrap Server is "bootstrap.wadmp.com"
BS2 = f"bootstrap.{url.netloc}"
# and the Management Server is "management.wadmp.com"
MS2 = f"management.{url.netloc}"

## Login to Server 1

In [ ]:
user1_token = login(GATEWAY1, SESSION1, USERNAME1, PASSWORD1)
SESSION1.headers.update({'Authorization': f'Bearer {user1_token}'})

## Check that the device exists on Server 1

In [ ]:
device = get_devices(GATEWAY1, SESSION1, 1, 10, device_name)[0]

## Login to Server 2

In [ ]:
user2_token = login(GATEWAY2, SESSION2, USERNAME2, PASSWORD2)
SESSION2.headers.update({'Authorization': f'Bearer {user2_token}'})

## Create the device on Server 2

In [ ]:
# Check if it exists already
try:
    device2 = get_devices(GATEWAY2, SESSION2, 1, 10, device_name)[0]
except IndexError:
    print(f"Device {device_name} does not exist on Server 2. Creating it ...")
    model = {
      "alias": device['alias'],
      "serial_number": device['serial_number'],
      "mac_address": device['mac_address'],
      "device_type_id": device['device_type']['type_id'],
      "order_code": device['order_code'],
      "imei": device['imei']
    }
    create_device(GATEWAY2, SESSION2, model=model)
else:
    print(f"Device {device_name} already exists on Server 2")

## Check what Bootstrap Server the device is using

In [ ]:
client_config = get_app_configuration(GATEWAY1, SESSION1, device['mac_address'], 'wadmp_client')
bs_url = ini_to_dict(client_config['reported_configuration'])['MOD_WADMP_CLIENT_BOOTSTRAPSERVER']
print(f"This device is configured to use {bs_url} as a Bootstrap Server")

In [ ]:
# Usually, the device's Bootstrap Server is Server 1 or Server 2, but it may not be!
# We refer to it as Server 0.
if bs_url == BS1:
    print("The Bootstrap Server is on Server 1")
    MS0 = MS1
    GATEWAY0 = GATEWAY1
    SESSION0 = SESSION1
    user0_token = user1_token
elif bs_url == BS2:
    print("The Bootstrap Server is on Server 2")
    MS0 = MS2
    GATEWAY0 = GATEWAY2
    SESSION0 = SESSION2
    user0_token = user2_token
else:
    print("The Bootstrap Server is on a third server, on which you must have credentials ...")
    USERNAME0 = input("Username of SysAdmin on Bootstrap Server:")
    PASSWORD0 = input("Password on Bootstrap Server:")

    # If bs_url is "bootstrap.dev.wadmp.com", the URL for the API Gateway is "https://gateway.dev.wadmp.com"
    netloc = bs_url.replace('bootstrap', 'gateway')
    GATEWAY0 = f"{SCHEME}://{netloc}"
    # and the Management Server is "management.dev.wadmp.com"
    MS0 = bs_url.replace('bootstrap', 'management')

    SESSION0 = requests.Session()  # Use one HTTPS session for all API calls to Bootstrap Server
    user0_token = login(GATEWAY0, SESSION0, USERNAME0, PASSWORD0)
    SESSION0.headers.update({'Authorization': f'Bearer {user0_token}'})

## Configure the Trust Store on Server 2
We take a conservative approach: any certificates in Server 1 that are NOT in Server 2 will be added to Server 2

First, check what certs are present in Server 2:

In [ ]:
certs2_b64 = get_certs(GATEWAY2, SESSION2)
certs2_pem = base64.b64decode(certs2_b64).decode('utf-8')  # b64decode() returns a "bytes-like" object

pattern = r"(-----BEGIN CERTIFICATE-----[A-Za-z0-9+/=\s]+-----END CERTIFICATE-----)"

certs2_list = re.findall(pattern, certs2_pem, re.MULTILINE)
certs2_dict = {}
print(f"{MS2} has {len(certs2_list)} certs:")
for cert_pem in certs2_list:
    # load_pem_x509_certificate() takes a "bytes-like" object
    cert_object = x509.load_pem_x509_certificate(cert_pem.encode('utf-8'), default_backend())
    print(f"   - {cert_object.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value}")
    certs2_dict[cert_object.subject] = cert_pem

Now, check what certs are present in Server 1:

In [ ]:
certs1_b64 = get_certs(GATEWAY1, SESSION1)
certs1_pem = base64.b64decode(certs1_b64).decode('utf-8')
new_certs2_pem = certs2_pem  # We will append to this string
update_required = False

certs1_list = re.findall(pattern, certs1_pem, re.MULTILINE)
print(f"{MS1} has {len(certs1_list)} certs:")
for cert_pem in certs1_list:
    cert_object = x509.load_pem_x509_certificate(cert_pem.encode('utf-8'), default_backend())
    print(f"   - {cert_object.subject.get_attributes_for_oid(NameOID.COMMON_NAME)[0].value}")
    if cert_object.subject in certs2_dict:
        print(f"     Also in {MS2}")
    else:
        print(f"     Not in {MS2} ... will be added!")
        new_certs2_pem += "\n" + cert_pem
        update_required = True

Now, add any missing certs:

In [ ]:
if update_required:
    new_certs2_b64 = base64.b64encode(new_certs2_pem.encode('utf-8')).decode('utf-8')  # b64encode() takes a "bytes-like" object and returns bytes
    put_certs(GATEWAY2, SESSION2, new_certs2_b64)
else:
    print("\nNo change required.")

## Find a Company ID on Server 0
We need a company ID later when creating a new Management Server definition on Server 0.
Our first choice is the company which has claimed the device.
If the device has not been claimed, we use one of the companies of which the user is a member.

In [ ]:
device = get_devices(GATEWAY0, SESSION0, 1, 10, device_name)[0]

if device['company']:
    company_id = device['company']['id']
else:
    claims = jwt.decode(user0_token, verify=False)
    company_ids = [key.split('.')[1] for key in claims.keys() if re.match(r"permissions\.\d+", key)]
    # We should probably check the permissions here, but for simplicity we just use the first company
    company_id = company_ids[0]

## Inform the Bootstrap Server (0) that Server 2 is available as a new Management Server

In [ ]:
ms_definitions = get_ms(GATEWAY0, SESSION0, Address=MS2)

if len(ms_definitions) == 0:
    print(f"We need to create {MS2} as a new MS definition on {MS0}")
    ms_id = create_ms(GATEWAY0, SESSION0, 'Server 2', MS2, 8883, company_id)
else:
    print(f"{MS2} is already defined on {MS0}")
    ms_definition = ms_definitions[0]
    ms_id = ms_definition['id']

## Configure the Bootstrap Server to use the new Management Server definition for this particular device

In [ ]:
use_ms(GATEWAY0, SESSION0, device['mac_address'], ms_id)

## Trigger the device to bootstrap again

In [ ]:
trigger_bootstrap(GATEWAY1, SESSION1, device['mac_address'])